# How to use ChatGPT API to build a chatbot for product recommendations with embeddings

**Are you looking to build a chatbot that can recommend products to your customers based on their unique profile?**

Here's a step-by-step guide that shows you how to build a chatbot using embeddings to match a user's profile with relevant products from a company's database.

Detailed step-by-step intructions for this repo in this blog post: https://norahsakal.com/blog/chatgpt-product-recommendation-embeddings

### Important
If you already have openai installed, make sure to call `pip install openai --upgrade` in your terminal to make sure you have access to version `0.27.0` (or higher) which has the newly added **gpt-3.5-turbo model**.

---

## Difference between ChatGPT and GPT-3 API
ChatGPT is OpenAI's new model family designed specifically for chat-based interactions.

Unlike the larger GPT-3 model, ChatGPT can consume a sequence of messages with metadata, which allows for more contextual understanding of conversations.

Plus, the ChatGPT API is currently priced at only $0.002 per 1k tokens, which is 10x cheaper than the existing GPT-3.5 models.

So if you're looking to build a chatbot for support requests, ChatGPT is definitely worth considering.

In [ ]:
import openai
import pandas as pd
from openai.embeddings_utils import get_embedding,cosine_similarity


# 1. Add your API key

In [ ]:
api_key ="YOUR_API_KEY"
openai.api_key = api_key

# 2. Create product data

In [ ]:

product_data = [{
    "prod_id": 1,
    "prod": "moisturizer",
    "brand":"Aveeno",
    "description": "for dry skin"
},
{
    "prod_id": 2,
    "prod": "foundation",
    "brand":"Maybelline",
    "description": "medium coverage"
},
{
    "prod_id": 3,
    "prod": "moisturizer",
    "brand":"CeraVe",
    "description": "for dry skin"
},
{
    "prod_id": 4,
    "prod": "nail polish",
    "brand":"OPI",
    "description": "raspberry red"
},
{
    "prod_id": 5,
    "prod": "concealer",
    "brand":"chanel",
    "description": "medium coverage"
},
{
    "prod_id": 6,
    "prod": "moisturizer",
    "brand":"Ole Henkrisen",
    "description": "for oily skin"
},
{
    "prod_id": 7,
    "prod": "moisturizer",
    "brand":"CeraVe",
    "description": "for normal to dry skin"
},
{
    "prod_id": 8,
    "prod": "moisturizer",
    "brand":"First Aid Beauty",
    "description": "for dry skin"
},{
    "prod_id": 9,
    "prod": "makeup sponge",
    "brand":"Sephora",
    "description": "super-soft, exclusive, latex-free foam"
}]

# 3. Add product data to dataframe

In [ ]:
product_data_df = pd.DataFrame(product_data)
product_data_df

# 4. Create column with combined data

In [ ]:
product_data_df['combined'] = product_data_df.apply(lambda row: f"{row['brand']}, {row['prod']}, {row['description']}", axis=1)
product_data_df

# 5. Create embeddings for combined product data

In [ ]:
product_data_df['text_embedding'] = product_data_df.combined.apply(lambda x: get_embedding(x, engine='text-embedding-ada-002'))
product_data_df

# 6. Create customer profile data

In [ ]:
customer_order_data = [
{
    "prod_id": 1,
    "prod": "moisturizer",
    "brand":"Aveeno",
    "description": "for dry skin"
},{
    "prod_id": 2,
    "prod": "foundation",
    "brand":"Maybelline",
    "description": "medium coverage"
},{
    "prod_id": 4,
    "prod": "nail polish",
    "brand":"OPI",
    "description": "raspberry red"
},{
    "prod_id": 5,
    "prod": "concealer",
    "brand":"chanel",
    "description": "medium coverage"
},{
    "prod_id": 9,
    "prod": "makeup sponge",
    "brand":"Sephora",
    "description": "super-soft, exclusive, latex-free foam"
}]

# 7. Add customer profile data to dataframe

In [ ]:
customer_order_df = pd.DataFrame(customer_order_data)
customer_order_df

# 8. Create combined column for customer profile

In [ ]:
customer_order_df['combined'] = customer_order_df.apply(lambda row: f"{row['brand']}, {row['prod']}, {row['description']}", axis=1)
customer_order_df


# 9. Create customer profile data embeddings

In [ ]:
customer_order_df['text_embedding'] = customer_order_df.combined.apply(lambda x: get_embedding(x, engine='text-embedding-ada-002'))
customer_order_df


# 10. Create embeddings for input question

In [ ]:
customer_input = "Hi! Can you recommend a good moisturizer for me?"

In [ ]:
response = openai.Embedding.create(
    input=customer_input,
    model="text-embedding-ada-002"
)
embeddings_customer_question = response['data'][0]['embedding']

# 11. Get similarities for purchase history

In [ ]:
customer_order_df['search_purchase_history'] = customer_order_df.text_embedding.apply(lambda x: cosine_similarity(x, embeddings_customer_question))
customer_order_df = customer_order_df.sort_values('search_purchase_history', ascending=False)
customer_order_df


# 12. Save top 3 similarities for purchase history

In [ ]:
top_3_purchases_df = customer_order_df.head(3)
top_3_purchases_df

# 13. Get similarities for products

In [ ]:
product_data_df['search_products'] = product_data_df.text_embedding.apply(lambda x: cosine_similarity(x, embeddings_customer_question))
product_data_df = product_data_df.sort_values('search_products', ascending=False)
product_data_df


# 14. Save top 3 similarities for products

In [ ]:
top_3_purchases_df = customer_order_df.head(3)
top_3_purchases_df

In [ ]:
top_3_products_df = product_data_df.head(3)
top_3_products_df

# 15. Create prompt

The system message helps set the behavior of the assistant.

>From OpenAI API docs: https://platform.openai.com/docs/guides/chat/introduction
>
>"gpt-3.5-turbo-0301 does not always pay strong attention to system messages. Future models will be trained to pay stronger attention to system messages."

>Tip 💡
>
>Tinker with the instructions in the prompt until you find the desired voice of your chatbot.

In [ ]:
message_objects = []
message_objects.append({"role":"system", "content":"You're a chatbot helping customers with beauty-realted questions and helping them with product recommendations"})


In [ ]:
# Append the customer messagae
message_objects.append({"role":"user", "content": customer_input})

In [ ]:
# Create previously purchased input
prev_purchases = ". ".join([f"{row['combined']}" for index, row in moisturizer_prev_purchases_df.iterrows()])
prev_purchases

In [ ]:
# Append prev relevant purchase
message_objects.append({"role":"user", "content": f"Here're my latest product orders: {prev_purchases}"})
message_objects.append({"role":"user", "content": f"Please give me a detailed explanation of your recommendations"})
message_objects.append({"role":"user", "content": "Please be friendly and talk to me like a person, don't just give me a list of recommentations"})


In [ ]:
# Create list of 3 products to recommend
products_list = []

for index, row in top_3_products_df.iterrows():
    brand_dict = {'role': "assistant", "content": f"{row['combined']}"}
    products_list.append(brand_dict)
products_list

In [ ]:
# Append found products  
message_objects.append({"role": "assistant", "content": f"I found these 3 products I would recommend"})
message_objects.extend(products_list)
message_objects.append({"role": "assistant", "content":"Here's my summarized recommendation of products, and why it would suit you:"})
message_objects

# 16. Call ChatGPT API

In [ ]:
completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=message_objects
)

print(completion.choices[0].message['content'])